In [ ]:
#pip install yfinance
#pip install PyPortfolioOpt

In [73]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pypfopt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

### Stock Selection

In [43]:
# Stock Selection
stock_list = ["D05.SI","Z74.SI","J36.SI","O39.SI","U11.SI","F34.SI","Y92.SI","Q0F.SI","BN4.SI","C38U.SI","C07.SI"]
#9CI.SI

In [45]:
START = '2015-01-01'
END = '2022-12-31'
df = pd.DataFrame()
period = "1y"
for company in stock_list:
    stock = yf.Ticker(company)
    hist = stock.history(start=START, end=END).Close
    df[company] = hist
df = df.dropna()
df

,D05.SI,Z74.SI,J36.SI,O39.SI,U11.SI,F34.SI,Y92.SI,Q0F.SI,BN4.SI,C38U.SI,C07.SI
Date,,,,,,,,,,,
2015-01-02 00:00:00+08:00,13.822885,2.709047,45.657066,7.215528,17.721083,2.431841,0.002571,1.414135,6.068272,1.303257,30.024017
2015-01-05 00:00:00+08:00,13.512717,2.702082,45.305981,7.112449,17.323425,2.402094,0.002516,1.402575,5.951838,1.309645,29.306086
2015-01-06 00:00:00+08:00,13.371113,2.674226,44.693432,6.981880,16.860693,2.372347,0.002516,1.379456,5.698422,1.290480,28.645576
2015-01-07 00:00:00+08:00,13.350887,2.688154,44.327393,6.995623,16.947458,2.357473,0.002607,1.379456,5.718971,1.328811,28.523529
2015-01-08 00:00:00+08:00,13.539682,2.743867,44.484276,7.215528,17.366808,2.409531,0.002607,1.398722,5.801158,1.296868,28.645576
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23 00:00:00+08:00,32.153221,2.490043,48.404598,11.612528,29.245790,3.981184,0.661024,1.662064,7.003848,1.916278,27.775372
2022-12-27 00:00:00+08:00,32.398808,2.499732,48.872322,11.574977,29.274183,3.981184,0.661024,1.670952,6.984816,1.916278,27.794725
2022-12-28 00:00:00+08:00,32.304352,2.499732,48.614597,11.593753,29.302576,4.038605,0.665849,1.653176,6.946752,1.916278,27.475357


In [51]:
mu = mean_historical_return(df)
S = CovarianceShrinkage(df).ledoit_wolf()

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(cleaned_weights)

ef.portfolio_performance(verbose=True)


OrderedDict([('D05.SI', 0.42802), ('Z74.SI', 0.0), ('J36.SI', 0.0), ('O39.SI', 0.01121), ('U11.SI', 0.02957), ('F34.SI', 0.10248), ('Y92.SI', 0.42871), ('Q0F.SI', 0.0), ('BN4.SI', 0.0), ('C38U.SI', 0.0), ('C07.SI', 0.0)])
Expected annual return: 48.8%
Annual volatility: 42.9%
Sharpe Ratio: 1.09


(0.48799373542964397, 0.4291055295832611, 1.0906262053629332)

In [60]:
stock_weights = list(cleaned_weights.values())
print(stock_weights)

[0.42802, 0.0, 0.0, 0.01121, 0.02957, 0.10248, 0.42871, 0.0, 0.0, 0.0, 0.0]


In [53]:
log_returns = np.log(df/df.shift(1))
log_returns

,D05.SI,Z74.SI,J36.SI,O39.SI,U11.SI,F34.SI,Y92.SI,Q0F.SI,BN4.SI,C38U.SI,C07.SI
Date,,,,,,,,,,,
2015-01-02 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05 00:00:00+08:00,-0.022694,-0.002574,-0.007719,-0.014389,-0.022695,-0.012308,-0.021662,-0.008208,-0.019374,0.004890,-0.024202
2015-01-06 00:00:00+08:00,-0.010535,-0.010363,-0.013612,-0.018528,-0.027075,-0.012461,0.000000,-0.016621,-0.043511,-0.014742,-0.022796
2015-01-07 00:00:00+08:00,-0.001514,0.005195,-0.008224,0.001966,0.005133,-0.006289,0.035846,0.000000,0.003600,0.029270,-0.004270
2015-01-08 00:00:00+08:00,0.014042,0.020513,0.003533,0.030951,0.024443,0.021842,0.000000,0.013870,0.014269,-0.024332,0.004270
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23 00:00:00+08:00,-0.006734,-0.007752,0.002567,0.004862,-0.008379,-0.002401,0.007326,0.027102,-0.005420,0.000000,-0.002784
2022-12-27 00:00:00+08:00,0.007609,0.003883,0.009616,-0.003239,0.000970,0.000000,0.000000,0.005333,-0.002721,0.000000,0.000697
2022-12-28 00:00:00+08:00,-0.002920,0.000000,-0.005287,0.001621,0.000969,0.014320,0.007273,-0.010695,-0.005465,0.000000,-0.011557


In [61]:
def calculate_returns(weights, returns):
    return np.sum(returns.mean() * weights) * 252

In [64]:
calculate_returns(stock_weights, log_returns)

0.3524154860459383

In [65]:
log_returns_cov = log_returns.cov()
log_returns_cov

,D05.SI,Z74.SI,J36.SI,O39.SI,U11.SI,F34.SI,Y92.SI,Q0F.SI,BN4.SI,C38U.SI,C07.SI
D05.SI,0.000150,0.000068,0.000050,0.000107,0.000112,0.000069,0.000075,0.000025,0.000092,0.000059,0.000085
Z74.SI,0.000068,0.000131,0.000036,0.000064,0.000064,0.000059,0.000053,0.000019,0.000069,0.000057,0.000070
J36.SI,0.000050,0.000036,0.000266,0.000048,0.000053,0.000044,0.000051,0.000021,0.000051,0.000045,0.000106
O39.SI,0.000107,0.000064,0.000048,0.000120,0.000103,0.000073,0.000050,0.000024,0.000092,0.000060,0.000085
U11.SI,0.000112,0.000064,0.000053,0.000103,0.000131,0.000068,0.000055,0.000020,0.000092,0.000056,0.000086
F34.SI,0.000069,0.000059,0.000044,0.000073,0.000068,0.000208,0.000043,0.000020,0.000088,0.000060,0.000079
Y92.SI,0.000075,0.000053,0.000051,0.000050,0.000055,0.000043,0.002011,0.000031,0.000078,0.000072,0.000084
Q0F.SI,0.000025,0.000019,0.000021,0.000024,0.000020,0.000020,0.000031,0.000207,0.000030,0.000018,0.000023
BN4.SI,0.000092,0.000069,0.000051,0.000092,0.000092,0.000088,0.000078,0.000030,0.000220,0.000057,0.000098
C38U.SI,0.000059,0.000057,0.000045,0.000060,0.000056,0.000060,0.000072,0.000018,0.000057,0.000183,0.000072


In [67]:
def calculate_volatility(weights, returns_covariance):
    cov_dot_weights = np.dot(returns_covariance * 252, weights)
    variance = np.dot(weights, cov_dot_weights)
    return np.sqrt(variance)


In [71]:
volatility = calculate_volatility(stock_weights, log_returns_cov)
print(volatility)

0.33418117964975425


In [72]:
risk_free_rate = 0

# Calculate the expected portfolio return
portfolio_return = calculate_returns(stock_weights, log_returns)

# Calculate the portfolio volatility
portfolio_volatility = calculate_volatility(stock_weights, log_returns_cov)

# Calculate the Sharpe ratio
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

print("Portfolio Return:", portfolio_return)
print("Portfolio Volatility:", portfolio_volatility)
print("Sharpe Ratio:", sharpe_ratio)


Portfolio Return: 0.3524154860459383
Portfolio Volatility: 0.33418117964975425
Sharpe Ratio: 1.0545641331905493


In [74]:
portfolio_value = 1000000

# Calculate the number of shares to purchase using DiscreteAllocation
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
allocation, leftover = da.lp_portfolio()

print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'Z74.SI': 20979, 'J36.SI': 458, 'O39.SI': 3741, 'U11.SI': 1971, 'F34.SI': 32591, 'Y92.SI': 728996, 'Q0F.SI': 38194, 'BN4.SI': 5114, 'C38U.SI': 30839, 'C07.SI': 1219}
Funds remaining: $20605.00


c:\Users\story\AppData\Local\Programs\Python\Python39\lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
